In [1]:
import pandas as pd

In [2]:
data_url = "https://raw.githubusercontent.com/fenago/tf/main/Chapter6-Regularization_and_Hyperparameter_Tuning/dataset/shuttle.trn"

data = pd.read_table(data_url, header=None, sep=' ')
data.head()

,0,1,2,3,4,5,6,7,8,9
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1
4,37,0,79,0,34,-26,43,46,2,1


In [3]:
y = data.pop(9)
X = data.copy()

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y)

In [7]:
import keras_tuner as kt

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [8]:
tf.random.set_seed(8)

In [9]:
def model_builder(hp):
        model = tf.keras.Sequential()
        hp_units = hp.Int('units', min_value=128, max_value=512, \
                          step=64)
        reg_fc1 = Dense(hp_units, input_shape=(9,), \
                        activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=0.0001))
        reg_fc2 = Dense(512, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=0.0001))
        reg_fc3 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=0.0001))
        reg_fc4 = Dense(128, activation='relu', \
                        kernel_regularizer=tf.keras.regularizers\
                                             .l2(l=0.0001))
        reg_fc5 = Dense(8, activation='softmax')
        model.add(reg_fc1)
        model.add(reg_fc2)
        model.add(reg_fc3)
        model.add(reg_fc4)
        model.add(reg_fc5)
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        hp_learning_rate = hp.Choice('learning_rate', \
                                     values = [0.01, 0.001, 0.0001])
        optimizer = tf.keras.optimizers.Adam(hp_learning_rate)
        model.compile(optimizer = optimizer, loss = loss, \
                      metrics = ['accuracy'])
        return model

In [11]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', \
                            max_epochs=5, overwrite=True)

In [12]:
tuner.search(X_train, y_train, validation_data=(X_test, y_test))

Trial 10 Complete [00h 00m 13s]
val_accuracy: 0.9973946213722229

Best val_accuracy So Far: 0.9973946213722229
Total elapsed time: 00h 01m 24s
INFO:tensorflow:Oracle triggered exit


In [13]:
best_hps = tuner.get_best_hyperparameters()[0]

In [14]:
best_units = best_hps.get('units')
best_units

256

In [15]:
best_lr = best_hps.get('learning_rate')
best_lr

0.001

In [19]:
model = tuner.hypermodel.build(best_hps)
model.fit(X_train, y_train, epochs=5, \
              validation_data=(X_test, y_test))

Epoch 1/5
952/952 [==============================] - 3s 3ms/step - loss: 0.4206 - accuracy: 0.9781 - val_loss: 0.5081 - val_accuracy: 0.9800
Epoch 2/5
952/952 [==============================] - 3s 3ms/step - loss: 0.2781 - accuracy: 0.9913 - val_loss: 0.2035 - val_accuracy: 0.9959
Epoch 3/5
952/952 [==============================] - 2s 3ms/step - loss: 0.1463 - accuracy: 0.9956 - val_loss: 0.0727 - val_accuracy: 0.9969
Epoch 4/5
952/952 [==============================] - 2s 2ms/step - loss: 0.0804 - accuracy: 0.9968 - val_loss: 0.0880 - val_accuracy: 0.9970
Epoch 5/5
952/952 [==============================] - 2s 3ms/step - loss: 0.0876 - accuracy: 0.9939 - val_loss: 0.0522 - val_accuracy: 0.9967
